In [ ]:
%%writefile .env
OPENAI_API_KEY=""YOur key

Writing .env


In [ ]:
!pip install python-dotenv

In [ ]:
!pip install langchain openai

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv('.env')
# api_key = os.getenv('OPENAI_API_KEY')
# print(api_key)



True

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['old_html', 'new_html'],
    template = "You are a automation script developer"
               "find the element <div data-findid='main' class='divclass'> in the new sample part of html \n {new_html} by taking the reference of the old sample part of html:\n {old_html}"
               "The data-findid is a special attribute added to the element to know the correct element you will not find this in the new html"
               "Generate a xpath using the data-indexyz attribute only the xpath should be in the format //*[data-indexyz='value']"
               "Do not include any other text but only the xpath"
)
old_html = """
<section>
<span>
<div data-findid='main class='divclass'></div>
<span>
</section>
"""

new_html = """
<section>
<span>
<div data-findid='main' class='subclass' dataindexyz="12"></div>
<span>
</section>
"""

new_html
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt= prompt_template_name)
chain.run({"old_html": old_html, "new_html": new_html})

"\n\n//div[@data-indexyz='12']"

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.6)

llm("Give me a name of animal which starts with z")




'\n\nZebra'

# LLM CHAIN

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['animal'],
    template = "What kind of food does {animal} eat"
)


In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt= prompt_template_name)
chain.run("Zebra")

<ipython-input-46-e3273f0900e5>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("Zebra")


'\n\nZebras are herbivores and primarily eat grass and other plant materials. They also eat leaves, twigs, bark, and fruits. In captivity, they may also be fed hay, grains, and vegetables. '

# SimpleSequentialChain

In [ ]:
prompt_template_name = PromptTemplate(
    input_variables = ['animal'],
    template = "Give me one name for my {animal}"
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['name'],
    template = "suggest some food items for {name}. Return it as a comma seperated list."
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains= [name_chain, food_items_chain])
response = chain.run("Dog")
print(response)





1. Grilled chicken
2. Brown rice
3. Steamed vegetables
4. Baked fish
5. Quinoa salad
6. Roasted sweet potatoes
7. Whole wheat pasta
8. Lean ground turkey
9. Hummus and whole grain pita bread
10. Vegetable stir-fry
11. Lentil soup
12. Greek yogurt with fresh fruit
13. Cauliflower rice
14. Turkey and avocado wrap
15. Tofu and vegetable stir-fry.


# Sequential Chain

```
# This is formatted as code
```



In [ ]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables = ['animal'],
    template = "Give me one name for my {animal}"
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name,output_key="animal_name")

llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables =['animal_name'],
    template = "suggest some food items for {animal_name}."
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items,output_key="food_items")

In [ ]:
from langchain.chains import SequentialChain

sequential_chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['animal'],
    output_variables = ["animal_name","food_items"],
    verbose =True

)

response = sequential_chain({"animal":"dog"})
print(response)

<ipython-input-50-c9c2904e404e>:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = sequential_chain({"animal":"dog"})




> Entering new SequentialChain chain...

> Finished chain.
{'animal': 'dog', 'animal_name': '\n\nBuddy', 'food_items': '\n\n1. Peanut butter and banana sandwich\n2. Apple slices and cheddar cheese\n3. Scrambled eggs and toast\n4. Chicken and vegetable stir fry\n5. Oatmeal with berries and honey\n6. Tuna salad wrap\n7. Baked sweet potato fries\n8. Grilled cheese and tomato soup\n9. Veggie and hummus wrap\n10. Turkey and avocado wrap\n11. Quinoa and black bean bowl\n12. Greek yogurt with granola and fruit\n13. Baked salmon with roasted vegetables\n14. Turkey and cheese roll-ups\n15. Fruit and vegetable smoothie.'}


In [ ]:
from langchain.agents import AgentType, initialize_agent,load_tools

from langchain.llms import OpenAI

tools = load_tools(["llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

agent.run("When was Elon musk born? what is his age right now in 2024? ")

<ipython-input-51-b6bd9ad1be43>:7: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


'Elon Musk was born in 1971 and will be 53 years old in 2024.'

In [ ]:
from langchain.prompts import PromptTemplate
prompt = ("what kind of food does {animal} eat"
          "what kind of food does lion eat")
prompt_template_name = PromptTemplate(
    input_variables = ['animal'],
    # template = "What kind of food does {animal} eat"
    template = prompt
)

from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt= prompt_template_name)
chain.run("Zebra")

'\n\nZebra typically eat grass, leaves, and other plant matter. They are herbivores.\n\nLions are carnivores and primarily eat meat. They hunt and eat a wide variety of animals, including antelope, buffalo, zebras, and other small mammals. They may also scavenge and eat carrion.'

In [ ]:
import logging

logging.basicConfig(filename='app.log',level=logging.INFO, format='%(asctime)s -%(levelname)s -%(message)s')

logging.info("This is a message")


print("HI")

HI
